In [1]:
import findspark
findspark.init()
findspark.find()

import pyspark
findspark.find()

'C:\\spark\\spark-2.3.3-bin-hadoop2.7'

In [2]:
from pyspark import SparkContext
sc=SparkContext(master="local[3]")
print(sc)

<SparkContext master=local[3] appName=pyspark-shell>


In [3]:
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.appName("spark_alquilar").getOrCreate()

In [97]:
comprar_alquilar = spark.read.csv('./comprar_alquilar (1).csv', header='true', inferSchema='true', sep=',') 
comprar_alquilar.show()

+--------+--------------+----------+------------+-------+--------+------------+-----+-------+-------+
|ingresos|gastos_comunes|pago_coche|gastos_otros|ahorros|vivienda|estado_civil|hijos|trabajo|comprar|
+--------+--------------+----------+------------+-------+--------+------------+-----+-------+-------+
|    6000|          1000|         0|         600|  50000|  400000|           0|    2|      2|      1|
|    6745|           944|       123|         429|  43240|  636897|           1|    3|      6|      0|
|    6455|          1033|        98|         795|  57463|  321779|           2|    1|      8|      1|
|    7098|          1278|        15|         254|  54506|  660933|           0|    0|      3|      0|
|    6167|           863|       223|         520|  41512|  348932|           0|    0|      3|      1|
|    5692|           911|        11|         325|  50875|  360863|           1|    4|      5|      1|
|    6830|          1298|       345|         309|  46761|  429812|           1|   

In [98]:
comprar_alquilar.dtypes 

[('ingresos', 'int'),
 ('gastos_comunes', 'int'),
 ('pago_coche', 'int'),
 ('gastos_otros', 'int'),
 ('ahorros', 'int'),
 ('vivienda', 'int'),
 ('estado_civil', 'int'),
 ('hijos', 'int'),
 ('trabajo', 'int'),
 ('comprar', 'int')]

In [99]:
import pyspark.ml.feature as ft
featuresCreator = ft.VectorAssembler(
    inputCols=['ingresos','ahorros','vivienda','hijos'], 
    outputCol='features'
)
output = featuresCreator.transform(comprar_alquilar)

In [100]:
from pyspark.ml.classification import NaiveBayes

In [101]:
nb=cl.NaiveBayes(featuresCol='features', labelCol='comprar')

In [109]:
data_train, data_test = comprar_alquilar.randomSplit([0.8, 0.2])

In [110]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[featuresCreator, nb])
model = pipeline.fit(data_train)

In [111]:
test_model = model.transform(data_test)

In [131]:
test_model.select('ingresos','ahorros','vivienda','hijos','prediction').show()

+--------+-------+--------+-----+----------+
|ingresos|ahorros|vivienda|hijos|prediction|
+--------+-------+--------+-----+----------+
|    2008|  25658|  180374|    3|       1.0|
|    2028|  37309|  284984|    0|       1.0|
|    2064|  38718|  356201|    0|       0.0|
|    2151|  12203|  339143|    1|       0.0|
|    2265|  19218|  248465|    0|       0.0|
|    2448|  32723|  362523|    0|       0.0|
|    2485|  18464|  304018|    2|       0.0|
|    2516|  16649|  262250|    0|       0.0|
|    2614|  21365|  294601|    0|       0.0|
|    2937|  13268|  177774|    1|       0.0|
|    3000|  27572|  254359|    0|       0.0|
|    3162|  26270|  216440|    2|       1.0|
|    3218|  39174|  330991|    1|       1.0|
|    3334|  19985|  376892|    2|       0.0|
|    3543|  30918|  327691|    4|       0.0|
|    3731|  39448|  349870|    0|       0.0|
|    3796|  35445|  210261|    0|       1.0|
|    3976|  11697|  218574|    0|       0.0|
|    4054|  22183|  223971|    0|       0.0|
|    4276|

In [140]:
df = spark.createDataFrame([
        (2000, 5000, 200000,0),
        (6000, 34000, 320000,2),], ['ingresos','ahorros','vivienda','hijos'])
df.show()

+--------+-------+--------+-----+
|ingresos|ahorros|vivienda|hijos|
+--------+-------+--------+-----+
|    2000|   5000|  200000|    0|
|    6000|  34000|  320000|    2|
+--------+-------+--------+-----+



In [136]:
prediccion = model.transform(df)

In [139]:
prediccion.select("prediction").show()

+----------+
|prediction|
+----------+
|         0|
|         1|
+----------+

